In [1]:
from toy_model import *
wandb.login()

/opt/anaconda3/envs/toytrans/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


True

In [2]:
T0 = np.array([
    [0, 1, 0],
    [0, 0, 1],
    [0, 0, 0.5]
])
T1 = np.array([
    [0, 0, 0],
    [0, 0, 0],
    [0.5, 0, 0]
])
dataset = MarkovData(10000, 30, 3, 2, [T0, T1])

In [3]:
#get bigram stats
stat_data=MarkovData(1000, 300, 3, 2, [T0, T1],seed=40)
x=torch.stack(stat_data.data)
# Count bigrams
bigram_counts = {'00': 0, '01': 0, '10': 0, '11': 0}
total_bigrams = 0

for i in range(x.shape[0]):  # for each sequence
    for j in range(x.shape[1] - 1):  # for each position (except last)
        bigram = f"{x[i, j].item()}{x[i, j+1].item()}"
        if bigram in bigram_counts:
            bigram_counts[bigram] += 1
        total_bigrams += 1

print(f"\nBigram frequencies:")
for pattern, count in bigram_counts.items():
    freq = count / total_bigrams
    print(f"{pattern}: {count:,} occurrences ({freq:.4f} = {freq*100:.2f}%)")

print(f"\nTotal bigrams: {total_bigrams:,}")
print(f"Expected total: {x.shape[0] * (x.shape[1] - 1):,}")


Bigram frequencies:
00: 149,392 occurrences (0.4996 = 49.96%)
01: 74,847 occurrences (0.2503 = 25.03%)
10: 74,761 occurrences (0.2500 = 25.00%)
11: 0 occurrences (0.0000 = 0.00%)

Total bigrams: 299,000
Expected total: 299,000


In [4]:
def get_trigram_stats(x):
    """
    Compute trigram frequencies from sequence data.
    
    Args:
        x: Tensor of shape (batch_size, sequence_length) containing integer sequences
    
    Returns:
        dict: Trigram frequencies and counts
    """
    trigram_counts = {
        '000': 0, '001': 0, '010': 0, '011': 0,
        '100': 0, '101': 0, '110': 0, '111': 0
    }
    total_trigrams = 0
    
    for i in range(x.shape[0]):  # for each sequence
        for j in range(x.shape[1] - 2):  # for each position (except last 2)
            trigram = f"{x[i, j].item()}{x[i, j+1].item()}{x[i, j+2].item()}"
            if trigram in trigram_counts:
                trigram_counts[trigram] += 1
            total_trigrams += 1
    
    # Convert to frequencies
    trigram_freqs = {}
    for pattern, count in trigram_counts.items():
        freq = count / total_trigrams if total_trigrams > 0 else 0
        trigram_freqs[pattern] = freq
    
    print(f"\nTrigram frequencies:")
    for pattern, count in trigram_counts.items():
        freq = trigram_freqs[pattern]
        print(f"{pattern}: {count:,} occurrences ({freq:.4f} = {freq*100:.2f}%)")
    print(f"\nTotal trigrams: {total_trigrams:,}")
    print(f"Expected total: {x.shape[0] * (x.shape[1] - 2):,}")
    
    return trigram_freqs, trigram_counts, total_trigrams


In [5]:
stat_data=MarkovData(1000, 300, 3, 2, [T0, T1],seed=40)
x=torch.stack(stat_data.data)
trigram_freqs, trigram_counts, total_trigrams = get_trigram_stats(x)


Trigram frequencies:
000: 74,305 occurrences (0.2493 = 24.93%)
001: 74,584 occurrences (0.2503 = 25.03%)
010: 74,609 occurrences (0.2504 = 25.04%)
011: 0 occurrences (0.0000 = 0.00%)
100: 74,502 occurrences (0.2500 = 25.00%)
101: 0 occurrences (0.0000 = 0.00%)
110: 0 occurrences (0.0000 = 0.00%)
111: 0 occurrences (0.0000 = 0.00%)

Total trigrams: 298,000
Expected total: 298,000


In [ ]:
# Metric Codes

def bigram_kl(model):
    T0 = np.array([
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 0.5]
    ])
    T1 = np.array([
        [0, 0, 0],
        [0, 0, 1],
        [0.5, 0, 0]
    ])

    # Generate test data
    test_data = MarkovData(50, 30, 3, 2, [T0, T1], seed=42)
    x = torch.stack(test_data.data)  # shape: (50, 30)

    # Build dist1 manually based on x values
    dist1 = torch.zeros(x.size(0), x.size(1), 2)  # shape: (50, 30, 2)
    dist1[..., 0] = torch.where(x == 1, 1, 0.5)  # P(0)
    dist1[..., 1] = torch.where(x == 1, 0, 0.5)  # P(1)

    # Get predicted probabilities from model
    dist2 = model(x).softmax(dim=-1)  # shape: (50, 30, 2)

    # Avoid log(0) by clamping very small values
    eps = 1e-8
    dist1_clamped = dist1.clamp(min=eps)
    dist2_clamped = dist2.clamp(min=eps)

    # Compute KL divergence manually: sum_i P(i) * log(P(i)/Q(i))
    def kl(dist1_clamped, dist2_clamped):
        kl = dist1_clamped * (dist1_clamped.log() - dist2_clamped.log())  # shape: (4, 10, 2)
        kl_sum = kl.sum(dim=-1)  # sum over distribution axis → shape: (4, 10)
        return kl_sum.mean().item()  # scalar

    return kl(dist1_clamped, dist2_clamped), kl(dist2_clamped, dist1_clamped)

def trigram_kl(model):
    """
    Compute KL divergence between true Mealy process and model predictions
    based on trigram-level distributions.
    """
    T0 = np.array([
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 0.5]
    ])
    T1 = np.array([
        [0, 0, 0],
        [0, 0, 0],
        [0.5, 0, 0]
    ])
    
    # Generate test data
    test_data = MarkovData(50, 32, 3, 2, [T0, T1], seed=42)
    x = torch.stack(test_data.data)  # shape: (50, 32)
    
    batch_size, seq_len = x.shape
    dist1 = torch.zeros(batch_size, seq_len, 2)  # shape: (50, 32, 2)
    
    # For trigram-based prediction, we need the last two tokens to predict the next
    for i in range(batch_size):
        for j in range(2, seq_len):  
            prev_bigram = f"{x[i, j-2].item()}{x[i, j-1].item()}"
            
            if prev_bigram == "00":
                dist1[i, j, 0] = 0.5    # P(0|00) 
                dist1[i, j, 1] = 0.5    # P(1|00)
            elif prev_bigram == "01":
                dist1[i, j, 0] = 1.0    # P(0|01)
                dist1[i, j, 1] = 0.0    # P(1|01)
            elif prev_bigram == "10":
                dist1[i, j, 0] = 1.0    # P(0|10)
                dist1[i, j, 1] = 0.0    # P(1|10)
            elif prev_bigram == "11":
                # This should never occur in the true process
                dist1[i, j, 0] = 0.5    # Fallback
                dist1[i, j, 1] = 0.5
    
    # Get predicted probabilities from model
    dist2 = model(x).softmax(dim=-1)  # shape: (50, 32, 2)
    
    # Only evaluate positions where we have trigram context (positions 2 and beyond)
    dist1_eval = dist1[:, 2:, :]
    dist2_eval = dist2[:, 2:, :]
    
    # Avoid log(0) by clamping very small values
    eps = 1e-8
    dist1_clamped = dist1_eval.clamp(min=eps)
    dist2_clamped = dist2_eval.clamp(min=eps)
    
    # Compute KL divergence manually: sum_i P(i) * log(P(i)/Q(i))
    def kl(dist1_clamped, dist2_clamped):
        kl_div = dist1_clamped * (dist1_clamped.log() - dist2_clamped.log())
        kl_sum = kl_div.sum(dim=-1)  # sum over distribution axis
        return kl_sum.mean().item()  # scalar
    
    return kl(dist1_clamped, dist2_clamped), kl(dist2_clamped, dist1_clamped)

def markov_kl(model):
    T0 = np.array([
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 0.5]
    ])
    T1 = np.array([
        [0, 0, 0],
        [0, 0, 1],
        [0.5, 0, 0]
    ])

    # Generate test data
    test_data = MarkovData(50, 30, 3, 2, [T0, T1], seed=42)
    x = torch.stack(test_data.data)  # shape: (4, 10)
    dist1 = []
    for etas in test_data.states:
        dist1.append([test_data.model.token_probabilities(eta) for eta in etas])
    dist1 = torch.tensor(np.array(dist1))[:, 1:, :]

    # Get predicted probabilities from model
    dist2 = model(x).softmax(dim=-1)  # shape: (4, 10, 2)

    # Avoid log(0) by clamping very small values
    eps = 1e-8
    dist1_clamped = dist1.clamp(min=eps)
    dist2_clamped = dist2.clamp(min=eps)

    # Compute KL divergence manually: sum_i P(i) * log(P(i)/Q(i))
    def kl(dist1_clamped, dist2_clamped):
        kl = dist1_clamped * (dist1_clamped.log() - dist2_clamped.log())  # shape: (4, 10, 2)
        kl_sum = kl.sum(dim=-1)  # sum over distribution axis → shape: (4, 10)
        return kl_sum.mean().item()  # scalar

    return kl(dist1_clamped, dist2_clamped), kl(dist2_clamped, dist1_clamped)

def test_on_all(model, gen_len, start=2):
    assert gen_len % 3 == 0, 'gen_len should be a multiple of 3'
    # Building the test data
    test_seq = [0, 1, None] * (gen_len // 3)
    all_seq = [test_seq]
    temp = []
    for i in range(2, len(test_seq), 3):
        for j in range(len(all_seq)):
            temp.append(all_seq[j].copy())
            temp[-1][i] = 0
            temp.append(all_seq[j].copy())
            temp[-1][i] = 1
        all_seq, temp = temp, []
    all_seq = [all_seq, [i[-1:] + i[:-1] for i in all_seq], [i[-2:] + i[:-2] for i in all_seq]]
    all_seq = [torch.tensor(seq, dtype=torch.int64) for seq in all_seq]
    
    # Testing the model
    errors = []
    acc = 0.0
    for i in range(3):
        preds = model(all_seq[i])[:,:-1,:].argmax(dim=-1)
        err = torch.where(all_seq[i][:,1:] != preds, 1, 0)
        for j in range(gen_len // 3):
            err[:, (3*j + i + 1) % (gen_len - 1)] = 0
        err[:,:start] = 0
        errors.append(err)
        acc += (1 - err.sum() / (err.shape[0] * err.shape[1])) * 100
    acc /= 3
    print(f'Accuracy: {acc:.2f} %')
    return torch.cat([all_seq[i][errors[i].sum(dim=-1).nonzero().flatten().tolist()] for i in range(3)])

# A1 - Base Model

In [7]:
model = train_model(
            dataset=dataset,
            n_epochs=50,
            n_layers=1,
            batch_size=64,
            d_model=16,
            attn_only=True,
            lr=0.1,
            wandb=True,
            wandb_project_name='ICL',
            save_dir='A1/proc1/',
            save_every=5
        )
wandb.finish()

Moving model to device:  cpu


  2%|▏         | 1/50 [00:00<00:21,  2.28it/s]

Epoch 1 Samples 8000 Step 124 Training Loss 0.4930770993232727
Epoch 1 Validation Loss 0.49595534801483154


  4%|▍         | 2/50 [00:00<00:18,  2.58it/s]

Epoch 2 Samples 8000 Step 124 Training Loss 0.4825283885002136
Epoch 2 Validation Loss 0.48329704999923706


...

 98%|█████████▊| 49/50 [00:20<00:00,  2.38it/s]

Epoch 49 Samples 8000 Step 124 Training Loss 0.4067683815956116
Epoch 49 Validation Loss 0.39814648032188416


100%|██████████| 50/50 [00:20<00:00,  2.44it/s]
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Epoch 50 Samples 8000 Step 124 Training Loss 0.39211350679397583
Epoch 50 Validation Loss 0.3948310315608978


epoch,▁▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇███████
samples,▄▅▂▅▇█▇▂▇▂▇▁▆▄▁█▂▃▂▁▆▁▃▄▇▆▇█▁▆▁▃▄▄▆▇▆▇▄█
train_loss,█▇▅▅▅▄▄▄▅▅▄▄▄▄▄▄▅▄▄▅▄▄▄▄▄▄▅▄▄▃▄▃▂▂▃▃▁▂▁▁
val_loss,█▇▇▆▆▆▆▆▆▆▆▅▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▁
epoch,50
samples,8000
train_loss,0.39211
val_loss,0.39483


In [9]:
model_name = 'A1'
print(model_name)
print('B||M = KL(Bigram || Model), M||B = KL(Model || Bigram)', end='\n\n')
for i in range(5, 300, 5):
    try:
        bm, mb = bigram_kl(load_model(f'{model_name}/model_{i}.pt', f'{model_name}/model_cfg.pt'))
        print(f"Model {i}: B||M - {bm:.3f}, M||B - {mb:.3f}")
    except Exception:
        break

A1
B||M = KL(Bigram || Model), M||B = KL(Model || Bigram)



In [65]:
model_name = 'A1'
print(model_name)
print('B||M = KL(Markov || Model), M||B = KL(Model || Markov)', end='\n\n')
for i in range(5, 300, 5):
    try:
        bm, mb = markov_kl(load_model(f'{model_name}/model_{i}.pt', f'{model_name}/model_cfg.pt'))
        print(f"Model {i}: B||M - {bm:.3f}, M||B - {mb:.3f}")
    except Exception:
        break

A1
B||M = KL(Markov || Model), M||B = KL(Model || Markov)

Model 5: B||M - 0.391, M||B - 4.544
Model 10: B||M - 0.389, M||B - 4.520
Model 15: B||M - 0.388, M||B - 4.504
Model 20: B||M - 0.386, M||B - 4.478
Model 25: B||M - 0.377, M||B - 4.357
Model 30: B||M - 0.277, M||B - 2.963
Model 35: B||M - 0.131, M||B - 1.292
Model 40: B||M - 0.096, M||B - 0.959
Model 45: B||M - 0.075, M||B - 0.650
Model 50: B||M - 0.062, M||B - 0.563
Model 55: B||M - 0.058, M||B - 0.526
Model 60: B||M - 0.056, M||B - 0.497
Model 65: B||M - 0.068, M||B - 0.640
Model 70: B||M - 0.062, M||B - 0.578
Model 75: B||M - 0.066, M||B - 0.571
Model 80: B||M - 0.068, M||B - 0.553
Model 85: B||M - 0.081, M||B - 0.638
Model 90: B||M - 0.078, M||B - 0.581
Model 95: B||M - 0.084, M||B - 0.591
Model 100: B||M - 0.083, M||B - 0.580
Model 105: B||M - 0.084, M||B - 0.548
Model 110: B||M - 0.087, M||B - 0.557
Model 115: B||M - 0.098, M||B - 0.607
Model 120: B||M - 0.088, M||B - 0.555
Model 125: B||M - 0.091, M||B - 0.567
Model 130: 

In [66]:
model_name = 'A1'
epoch = 5
print(model_name, end='\n\n')
while True:
    try:
        print(f'Model_{epoch}')
        for i in test_on_all(load_model(f'{model_name}/model_{epoch}.pt', f'{model_name}/model_cfg.pt'), 9):
            print('|', end='')
            #print(f'Sequence: {i.tolist()}, Predictions: {model(i).argmax(dim=-1).flatten().tolist()}')
        print()
        epoch += 5
    except:
        break

A1

Model_5
Accuracy: 87.50 %
||||||||||||||||||
Model_10
Accuracy: 87.50 %
||||||||||||||||||
Model_15
Accuracy: 87.50 %
||||||||||||||||||
Model_20
Accuracy: 87.50 %
||||||||||||||||||
Model_25
Accuracy: 87.50 %
||||||||||||||||||
Model_30
Accuracy: 92.71 %
||||||||||||
Model_35
Accuracy: 95.83 %
||||||
Model_40
Accuracy: 96.88 %
||||||
Model_45
Accuracy: 96.88 %
||||
Model_50
Accuracy: 96.88 %
||||||
Model_55
Accuracy: 96.88 %
||||
Model_60
Accuracy: 96.88 %
||||||
Model_65
Accuracy: 96.88 %
||||||
Model_70
Accuracy: 96.88 %
||||||
Model_75
Accuracy: 96.88 %
||||
Model_80
Accuracy: 96.88 %
||||
Model_85
Accuracy: 96.88 %
||||||
Model_90
Accuracy: 96.88 %
||||
Model_95
Accuracy: 96.88 %
||||
Model_100
Accuracy: 96.88 %
||||
Model_105
Accuracy: 96.88 %
||||
Model_110
Accuracy: 96.88 %
||||
Model_115
Accuracy: 96.88 %
||||||
Model_120
Accuracy: 96.88 %
||||
Model_125
Accuracy: 96.88 %
||||||
Model_130
Accuracy: 96.88 %
||||
Model_135
Accuracy: 96.88 %
||||
Model_140
Accuracy: 96.88 %
|

# B1 - Layer Norm

In [77]:
model = train_model(
            dataset=dataset,
            n_epochs=300,
            n_layers=1,
            d_model=4,
            attn_only=True,
            lr=0.1,
            normalization_type='LNPre',
            wandb=True,
            wandb_project_name='superposition',
            save_dir='B1_noscale/',
            save_every=5
        )
wandb.finish()

Moving model to device:  cpu


  0%|          | 1/300 [00:04<20:10,  4.05s/it]

Epoch 1 Samples 8000 Step 124 Training Loss 0.6384778618812561
Epoch 1 Validation Loss 0.6402980089187622


  1%|          | 2/300 [00:07<17:11,  3.46s/it]

Epoch 2 Samples 8000 Step 124 Training Loss 0.6381754875183105
Epoch 2 Validation Loss 0.6335560083389282


...

100%|█████████▉| 299/300 [12:27<00:02,  2.38s/it]

Epoch 299 Samples 8000 Step 124 Training Loss 0.2816503643989563
Epoch 299 Validation Loss 0.2791288197040558


100%|██████████| 300/300 [12:29<00:00,  2.50s/it]
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Epoch 300 Samples 8000 Step 124 Training Loss 0.2786913514137268
Epoch 300 Validation Loss 0.27877387404441833


epoch,▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▇▇▇▇▇▇████
samples,▁▄▅▆▄▇▅▇█▃▇▇▂▃▁▅▁▅█▆▅▂▃▃▇▁▇▂█▃▂▄█▆▆▄▆▆▅▃
train_loss,█▅▅▄▄▃▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁
epoch,300
samples,8000
train_loss,0.27869
val_loss,0.27877


In [ ]:
model = train_model(
            dataset=dataset,
            n_epochs=25,
            d_model=4,
            attn_only=True,
            lr=0.2,
            normalization_type='LN',
            wandb=True,
            wandb_project_name='superposition',
            save_dir='B1_B',
            save_every=5
        )
wandb.finish()

Moving model to device:  cpu


  4%|▍         | 1/25 [00:06<02:43,  6.80s/it]

Epoch 1 Samples 8000 Step 124 Training Loss 0.604846179485321
Epoch 1 Validation Loss 0.5970064401626587


  8%|▊         | 2/25 [00:13<02:30,  6.56s/it]

Epoch 2 Samples 8000 Step 124 Training Loss 0.5687693953514099
Epoch 2 Validation Loss 0.5740916132926941


...

Epoch 23 Samples 8000 Step 124 Training Loss 0.2559430003166199
Epoch 23 Validation Loss 0.25720712542533875
Epoch 24 Samples 8000 Step 124 Training Loss 0.25245070457458496
Epoch 24 Validation Loss 0.256989449262619


 96%|█████████▌| 24/25 [02:38<00:06,  6.63s/it]

Epoch 25 Samples 8000 Step 124 Training Loss 0.25624406337738037
Epoch 25 Validation Loss 0.2598312199115753


100%|██████████| 25/25 [02:44<00:00,  6.59s/it]
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch,▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇██
samples,▁▃▅▃▆▃▃▂▅▁▅▁▄▇█▆▂▃▅▁▃▃▄▅▂▃▄▆▇▃▆▄▅▄█▇▄▆▆▄
train_loss,▇▇▆▆▇▆▄▃▆▃▃▅▂▂▂█▂▂▂▂▂▂▆▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▇███▃▃▂▂▂▂▁▇▂▁▁▁▁▁▁▁▁▁▁▁
epoch,25
samples,8000
train_loss,0.25624
val_loss,0.25983


In [ ]:
model_name = 'A1'
print(model_name)
print('B||M = KL(Bigram || Model), M||B = KL(Bigram || Markov)', end='\n\n')
for i in range(5, 300, 5):
    try:
        bm, mb = markov_kl(load_model(f'{model_name}/model_{i}.pt', f'{model_name}/model_cfg.pt'))
        print(f"Model {i}: B||M - {bm:.3f}, M||B - {mb:.3f}")
    except Exception:
        break

B1
B||M = KL(Bigram || Model), M||B = KL(Bigram || Markov)

Model 5: B||M - 0.383, M||B - 4.442
Model 10: B||M - 0.341, M||B - 3.899
Model 15: B||M - 0.292, M||B - 3.198
Model 20: B||M - 0.274, M||B - 3.012
Model 25: B||M - 0.246, M||B - 2.678
Model 30: B||M - 0.206, M||B - 2.282
Model 35: B||M - 0.304, M||B - 2.642
Model 40: B||M - 0.126, M||B - 1.281
Model 45: B||M - 0.106, M||B - 1.091
Model 50: B||M - 0.184, M||B - 1.548
Model 55: B||M - 0.082, M||B - 0.856
Model 60: B||M - 0.078, M||B - 0.818
Model 65: B||M - 0.072, M||B - 0.746
Model 70: B||M - 0.267, M||B - 2.807
Model 75: B||M - 0.098, M||B - 0.852
Model 80: B||M - 0.106, M||B - 0.742
Model 85: B||M - 0.114, M||B - 0.731
Model 90: B||M - 0.116, M||B - 0.725
Model 95: B||M - 0.121, M||B - 0.710
Model 100: B||M - 0.129, M||B - 0.729


In [58]:
model_name = 'B1'
print(model_name)
print('B||M = KL(Markov || Model), M||B = KL(Model || Markov)', end='\n\n')
for i in range(5, 300, 5):
    try:
        bm, mb = markov_kl(load_model(f'{model_name}/model_{i}.pt', f'{model_name}/model_cfg.pt'))
        print(f"Model {i}: B||M - {bm:.3f}, M||B - {mb:.3f}")
    except Exception:
        break

B1
B||M = KL(Markov || Model), M||B = KL(Model || Markov)

Model 5: B||M - 0.383, M||B - 4.442
Model 10: B||M - 0.341, M||B - 3.899
Model 15: B||M - 0.292, M||B - 3.198
Model 20: B||M - 0.274, M||B - 3.012
Model 25: B||M - 0.246, M||B - 2.678
Model 30: B||M - 0.206, M||B - 2.282
Model 35: B||M - 0.304, M||B - 2.642
Model 40: B||M - 0.126, M||B - 1.281
Model 45: B||M - 0.106, M||B - 1.091
Model 50: B||M - 0.184, M||B - 1.548
Model 55: B||M - 0.082, M||B - 0.856
Model 60: B||M - 0.078, M||B - 0.818
Model 65: B||M - 0.072, M||B - 0.746
Model 70: B||M - 0.267, M||B - 2.807
Model 75: B||M - 0.098, M||B - 0.852
Model 80: B||M - 0.106, M||B - 0.742
Model 85: B||M - 0.114, M||B - 0.731
Model 90: B||M - 0.116, M||B - 0.725
Model 95: B||M - 0.121, M||B - 0.710
Model 100: B||M - 0.129, M||B - 0.729


In [56]:
model_name = 'B1'
epoch = 5
print(model_name, end='\n\n')
while True:
    try:
        print(f'Model_{epoch}')
        for i in test_on_all(load_model(f'{model_name}/model_{epoch}.pt', f'{model_name}/model_cfg.pt'), 9):
            print('|', end='')
            #print(f'Sequence: {i.tolist()}, Predictions: {model(i).argmax(dim=-1).flatten().tolist()}')
        print()
        epoch += 5
    except:
        break

B1

Model_5
Accuracy: 87.50 %
||||||||||||||||||
Model_10
Accuracy: 88.54 %
|||||||||||||||||
Model_15
Accuracy: 89.06 %
||||||||||||||||
Model_20
Accuracy: 88.54 %
||||||||||||||||
Model_25
Accuracy: 91.67 %
|||||||||||||
Model_30
Accuracy: 97.92 %
||||
Model_35
Accuracy: 91.67 %
||||||||||||||||
Model_40
Accuracy: 97.40 %
|||||
Model_45
Accuracy: 97.40 %
|||||
Model_50
Accuracy: 91.67 %
||||||||||||
Model_55
Accuracy: 97.40 %
|||||
Model_60
Accuracy: 97.40 %
|||||
Model_65
Accuracy: 97.40 %
|||||
Model_70
Accuracy: 94.79 %
||||||||||
Model_75
Accuracy: 98.96 %
||
Model_80
Accuracy: 99.48 %
|
Model_85
Accuracy: 98.96 %
||
Model_90
Accuracy: 98.96 %
||
Model_95
Accuracy: 100.00 %

Model_100
Accuracy: 100.00 %

Model_105


# C1 - MLP No LN

In [86]:
model = train_model(
            dataset=dataset,
            n_epochs=300,
            n_layers=1,
            d_model=4,
            d_mlp=16,
            lr=0.1,
            wandb=True,
            wandb_project_name='superposition',
            save_every=5,
            save_dir='C1'
        )
wandb.finish()

Moving model to device:  cpu


  0%|          | 1/300 [00:01<08:06,  1.63s/it]

Epoch 1 Samples 8000 Step 124 Training Loss 0.6456921696662903
Epoch 1 Validation Loss 0.6470196843147278


  1%|          | 2/300 [00:03<07:50,  1.58s/it]

Epoch 2 Samples 8000 Step 124 Training Loss 0.6414846181869507
Epoch 2 Validation Loss 0.6419138312339783


...

100%|█████████▉| 299/300 [09:18<00:01,  1.93s/it]

Epoch 299 Samples 8000 Step 124 Training Loss 0.25705933570861816
Epoch 299 Validation Loss 0.2618131935596466


100%|██████████| 300/300 [09:20<00:00,  1.87s/it]
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Epoch 300 Samples 8000 Step 124 Training Loss 0.266920804977417
Epoch 300 Validation Loss 0.26216280460357666


epoch,▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▇▇▇▇▇▇██
samples,▇▃▂▇▁▆▆▃▃▃▂█▅▆█▂▇▅▃▂▅▄▁▆▆▃█▅▄▃▂▄▃▇▃▃▇▅▃▃
train_loss,█▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,████▇▄▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,300
samples,8000
train_loss,0.26692
val_loss,0.26216


In [109]:
print('B||M = KL(Bigram || Model), M||B = KL(Model || Bigram)')
for i in range(5, 100, 5):
    try:
        bm, mb = bigram_kl(load_model(f'B1_B/model_{i}.pt', 'B1/model_cfg.pt'))
        print(f"Model {i}: B||M - {bm:.3f}, M||B - {mb:.3f}")
    except:
        break

B||M = KL(Bigram || Model), M||B = KL(Model || Bigram)
Model 5: B||M - 0.077, M||B - 0.068
Model 10: B||M - 0.703, M||B - 0.335
Model 15: B||M - 1.068, M||B - 0.365
Model 20: B||M - 1.401, M||B - 0.386
Model 25: B||M - 1.567, M||B - 0.373


In [378]:
model_name = 'B1'
print(model_name)
print('B||M = KL(Markov || Model), M||B = KL(Model || Markov)', end='\n\n')
for i in range(5, 100, 5):
    try:
        bm, mb = markov_kl(load_model(f'{model_name}/model_{i}.pt', f'{model_name}/model_cfg.pt'))
        print(f"Model {i}: B||M - {bm:.3f}, M||B - {mb:.3f}")
    except Exception:
        break

B1
B||M = KL(Markov || Model), M||B = KL(Model || Markov)

Model 5: B||M - 0.375, M||B - 4.106
Model 10: B||M - 0.087, M||B - 0.747
Model 15: B||M - 0.082, M||B - 0.495
Model 20: B||M - 0.116, M||B - 0.375
Model 25: B||M - 0.146, M||B - 0.359
Model 30: B||M - 0.282, M||B - 2.836
Model 35: B||M - 0.178, M||B - 0.913
Model 40: B||M - 0.181, M||B - 0.845
Model 45: B||M - 0.173, M||B - 0.583


In [376]:
model_name = 'B1'
epoch = 5
print(model_name, end='\n\n')
while True:
    try:
        print(f'Model_{epoch}')
        for i in test_on_all(load_model(f'{model_name}/model_{epoch}.pt', f'{model_name}/model_cfg.pt'), 30):
            print('|', end='')
            #print(f'Sequence: {i.tolist()}, Predictions: {model(i).argmax(dim=-1).flatten().tolist()}')
        print()
        epoch += 5
    except:
        break

B1

Model_5
Accuracy: 87.22 %
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

# C1_

In [87]:
model = train_model(
            dataset=dataset,
            n_epochs=300,
            n_layers=1,
            d_model=2,
            d_mlp=16,
            lr=0.1,
            wandb=True,
            wandb_project_name='superposition',
            save_every=5,
            save_dir='C1_'
        )
wandb.finish()

Moving model to device:  cpu


  0%|          | 1/300 [00:01<06:30,  1.31s/it]

Epoch 1 Samples 8000 Step 124 Training Loss 0.6661055088043213
Epoch 1 Validation Loss 0.660175621509552


  1%|          | 2/300 [00:02<06:27,  1.30s/it]

Epoch 2 Samples 8000 Step 124 Training Loss 0.6472493410110474
Epoch 2 Validation Loss 0.6473472714424133


...

100%|█████████▉| 299/300 [08:33<00:02,  2.04s/it]

Epoch 299 Samples 8000 Step 124 Training Loss 0.38891327381134033
Epoch 299 Validation Loss 0.3617330491542816


100%|██████████| 300/300 [08:34<00:00,  1.72s/it]
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Epoch 300 Samples 8000 Step 124 Training Loss 0.3869464099407196
Epoch 300 Validation Loss 0.5135846138000488


epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇█████
samples,▄▄▇▇▅▆▃▁▆▁▇▂▆▆▁▆▄▅▅▂▄▆▇▂▇▄▇█▇▆▆▂▃▅▂▅▂▄▅█
train_loss,███▇▅▄▅▄▄▄▆▄▂▃▂▄▂▂▂▂▃▃▂▂▂▃▄▂▂▂▂▁▂▁▆▃▁▁▁▂
val_loss,███▅▅▅▄▄▄▄▃▂▂▂▂▂▂▁▂▆▁▂▂▂▃▁▅▁▂▁▁▁▁▁▂▁▁▁▁▄
epoch,300
samples,8000
train_loss,0.38695
val_loss,0.51358


# D1 - MLP + LN

In [98]:
model = train_model(
            dataset=dataset,
            n_epochs=300,
            n_layers=1,
            d_model=4,
            d_mlp=16,
            lr=0.05,
            normalization_type='LN',
            wandb=True,
            wandb_project_name='superposition',
            save_every=5,
            save_dir='D1'
        )
wandb.finish()

Moving model to device:  cpu


  0%|          | 1/300 [00:01<05:20,  1.07s/it]

Epoch 1 Samples 8000 Step 124 Training Loss 0.6242347955703735
Epoch 1 Validation Loss 0.6167828440666199


  1%|          | 2/300 [00:02<04:58,  1.00s/it]

Epoch 2 Samples 8000 Step 124 Training Loss 0.5873025059700012
Epoch 2 Validation Loss 0.5860589742660522


...

100%|█████████▉| 299/300 [07:25<00:01,  1.45s/it]

Epoch 299 Samples 8000 Step 124 Training Loss 0.2583736181259155
Epoch 299 Validation Loss 0.2592010498046875


100%|██████████| 300/300 [07:26<00:00,  1.49s/it]
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Epoch 300 Samples 8000 Step 124 Training Loss 0.26113560795783997
Epoch 300 Validation Loss 0.2589398920536041


epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇█
samples,▂▃▂▅█▂▂▁▅▆▇▁█▇▂▄▇▁▄▃▅▂▅▅▄▁▄▁▃▄▄▇█▄▃▄▁▂█▅
train_loss,██▇▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,300
samples,8000
train_loss,0.26114
val_loss,0.25894


In [99]:
model = train_model(
            dataset=dataset,
            n_epochs=300,
            n_layers=1,
            d_model=4,
            d_mlp=16,
            lr=0.05,
            normalization_type='LNPre',
            wandb=True,
            wandb_project_name='superposition',
            save_every=5,
            save_dir='D1_noscale'
        )
wandb.finish()

Moving model to device:  cpu


  0%|          | 1/300 [00:01<05:56,  1.19s/it]

Epoch 1 Samples 8000 Step 124 Training Loss 0.6370563507080078
Epoch 1 Validation Loss 0.6186138987541199


  1%|          | 2/300 [00:02<07:39,  1.54s/it]

Epoch 2 Samples 8000 Step 124 Training Loss 0.6218891143798828
Epoch 2 Validation Loss 0.619475781917572


...

100%|█████████▉| 299/300 [07:49<00:01,  1.51s/it]

Epoch 299 Samples 8000 Step 124 Training Loss 0.2619682550430298
Epoch 299 Validation Loss 0.2606712579727173


100%|██████████| 300/300 [07:51<00:00,  1.57s/it]
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Epoch 300 Samples 8000 Step 124 Training Loss 0.26109614968299866
Epoch 300 Validation Loss 0.25858303904533386


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇████
samples,▆▇▇▆▁▂█▇▄▄▆▂▁▇▄▆▂▅▂▁▄▆█▁▂▃▅▄▅▅▄▄▆▃▄▆▆▅▃▅
train_loss,█▄▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,300
samples,8000
train_loss,0.2611
val_loss,0.25858


In [ ]:
model_name = 'B1'
print(model_name)
print('B||M = KL(Markov || Model), M||B = KL(Model || Markov)', end='\n\n')
for i in range(5, 100, 5):
    try:
        bm, mb = markov_kl(load_model(f'{model_name}/model_{i}.pt', f'{model_name}/model_cfg.pt'))
        print(f"Model {i}: B||M - {bm:.3f}, M||B - {mb:.3f}")
    except Exception:
        break

B1
B||M = KL(Markov || Model), M||B = KL(Model || Markov)

Model 5: B||M - 0.375, M||B - 4.106
Model 10: B||M - 0.087, M||B - 0.747
Model 15: B||M - 0.082, M||B - 0.495
Model 20: B||M - 0.116, M||B - 0.375
Model 25: B||M - 0.146, M||B - 0.359
Model 30: B||M - 0.282, M||B - 2.836
Model 35: B||M - 0.178, M||B - 0.913
Model 40: B||M - 0.181, M||B - 0.845
Model 45: B||M - 0.173, M||B - 0.583


In [ ]:
model_name = 'B1'
epoch = 5
print(model_name, end='\n\n')
while True:
    try:
        print(f'Model_{epoch}')
        for i in test_on_all(load_model(f'{model_name}/model_{epoch}.pt', f'{model_name}/model_cfg.pt'), 30):
            print('|', end='')
            #print(f'Sequence: {i.tolist()}, Predictions: {model(i).argmax(dim=-1).flatten().tolist()}')
        print()
        epoch += 5
    except:
        break

B1

Model_5
Accuracy: 87.22 %
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

# D1_

In [102]:
model = train_model(
            dataset=dataset,
            n_epochs=300,
            n_layers=1,
            d_model=2,
            d_mlp=16,
            lr=0.05,
            normalization_type='LN',
            wandb=True,
            wandb_project_name='superposition',
            save_every=5,
            save_dir='D1_'
        )
wandb.finish()

Moving model to device:  cpu


  0%|          | 1/300 [00:00<04:37,  1.08it/s]

Epoch 1 Samples 8000 Step 124 Training Loss 0.687646746635437
Epoch 1 Validation Loss 0.6893687844276428


  1%|          | 2/300 [00:01<04:44,  1.05it/s]

Epoch 2 Samples 8000 Step 124 Training Loss 0.6901377439498901
Epoch 2 Validation Loss 0.6893411874771118


...

100%|█████████▉| 299/300 [06:56<00:01,  1.13s/it]

Epoch 299 Samples 8000 Step 124 Training Loss 0.6358596086502075
Epoch 299 Validation Loss 0.6359239220619202


100%|██████████| 300/300 [06:58<00:00,  1.39s/it]
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Epoch 300 Samples 8000 Step 124 Training Loss 0.635223925113678
Epoch 300 Validation Loss 0.6361018419265747


epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇███
samples,▅█▇▆█▃▆▆▇▃▆█▃▄▅▂▅▄▄▃▂▇▄▇▂▁▂▅▁▆▅█▃█▃█▄▄█▅
train_loss,██▇▇█▅▇▆▄▂▂▁▂▂▁▂▂▂▁▂▁▁▂▂▁▂▁▂▁▂▂▂▂▂▂▂▂▂▂▂
val_loss,██████▅▅▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,300
samples,8000
train_loss,0.63522
val_loss,0.6361


In [103]:
model = train_model(
            dataset=dataset,
            n_epochs=300,
            n_layers=1,
            d_model=2,
            d_mlp=16,
            lr=0.05,
            normalization_type='LNPre',
            wandb=True,
            wandb_project_name='superposition',
            save_every=5,
            save_dir='D1__noscale'
        )
wandb.finish()

Moving model to device:  cpu


  0%|          | 1/300 [00:01<06:05,  1.22s/it]

Epoch 1 Samples 8000 Step 124 Training Loss 0.6927733421325684
Epoch 1 Validation Loss 0.6925334334373474


  1%|          | 2/300 [00:02<05:40,  1.14s/it]

Epoch 2 Samples 8000 Step 124 Training Loss 0.6920993328094482
Epoch 2 Validation Loss 0.6925355195999146


...

100%|█████████▉| 299/300 [07:05<00:01,  1.36s/it]

Epoch 299 Samples 8000 Step 124 Training Loss 0.6899139881134033
Epoch 299 Validation Loss 0.6877452731132507


100%|██████████| 300/300 [07:07<00:00,  1.43s/it]
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Epoch 300 Samples 8000 Step 124 Training Loss 0.6833004355430603
Epoch 300 Validation Loss 0.687743604183197


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇████
samples,▆▇▃▅▄▅▂▂▃▄██▆▆▆▆▁▄▃▇▆▇▂▁▅▆▆▆▆▅▄▃▂▇▆▅▅▅▆▇
train_loss,▇▇██▇▇▆▇▇▇▇▇▆▇▇▇▇▆▆▇▆█▇▇▆▅▃▂▄▃▃▃▄▁▆▅▄▅▂▄
val_loss,█████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▅▂▂▁▁▁▁▁▁▁▁▁▁▁▁
epoch,300
samples,8000
train_loss,0.6833
val_loss,0.68774


In [ ]:
model_name = 'B1'
print(model_name)
print('B||M = KL(Markov || Model), M||B = KL(Model || Markov)', end='\n\n')
for i in range(5, 100, 5):
    try:
        bm, mb = markov_kl(load_model(f'{model_name}/model_{i}.pt', f'{model_name}/model_cfg.pt'))
        print(f"Model {i}: B||M - {bm:.3f}, M||B - {mb:.3f}")
    except Exception:
        break

B1
B||M = KL(Markov || Model), M||B = KL(Model || Markov)

Model 5: B||M - 0.375, M||B - 4.106
Model 10: B||M - 0.087, M||B - 0.747
Model 15: B||M - 0.082, M||B - 0.495
Model 20: B||M - 0.116, M||B - 0.375
Model 25: B||M - 0.146, M||B - 0.359
Model 30: B||M - 0.282, M||B - 2.836
Model 35: B||M - 0.178, M||B - 0.913
Model 40: B||M - 0.181, M||B - 0.845
Model 45: B||M - 0.173, M||B - 0.583


In [ ]:
model_name = 'B1'
epoch = 5
print(model_name, end='\n\n')
while True:
    try:
        print(f'Model_{epoch}')
        for i in test_on_all(load_model(f'{model_name}/model_{epoch}.pt', f'{model_name}/model_cfg.pt'), 30):
            print('|', end='')
            #print(f'Sequence: {i.tolist()}, Predictions: {model(i).argmax(dim=-1).flatten().tolist()}')
        print()
        epoch += 5
    except:
        break

B1

Model_5
Accuracy: 87.22 %
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||